Authors: Licong Xu and Boris Bolliet (Cambridge)

In [1]:
from pathlib import Path

from mcp import ClientSession, StdioServerParameters
from mcp.client.sse import sse_client
from mcp.client.stdio import stdio_client

from autogen import LLMConfig
from autogen.agentchat import AssistantAgent
from autogen.mcp import create_toolkit
import json
import anyio
import asyncio

# Only needed for Jupyter notebooks
import nest_asyncio
nest_asyncio.apply()

from autogen.agentchat.group import (
    AgentNameTarget,
    AgentTarget,
    AskUserTarget,
    ContextExpression,
    ContextStr,
    ContextStrLLMCondition,
    ContextVariables,
    ExpressionAvailableCondition,
    ExpressionContextCondition,
    GroupChatConfig,
    GroupChatTarget,
    Handoffs,
    NestedChatTarget,
    OnCondition,
    OnContextCondition,
    ReplyResult,
    RevertToUserTarget,
    SpeakerSelectionResult,
    StayTarget,
    StringAvailableCondition,
    StringContextCondition,
    StringLLMCondition,
    TerminateTarget,
)

from autogen.agentchat.group.patterns import (
    DefaultPattern,
    ManualPattern,
    AutoPattern,
    RandomPattern,
    RoundRobinPattern,
)


from autogen import ConversableAgent, UpdateSystemMessage
from autogen.agents.experimental import DocAgent
import os
import copy
from typing import Any, Dict, List
from pydantic import BaseModel, Field


from autogen.agentchat import initiate_group_chat, a_initiate_group_chat

In [2]:
# Path to the arxiv MCP server
mcp_server_path = Path("mcp_arxiv.py")


In [3]:
joker_message = """
You are the joker in the team. You make jokes in the style of shakespeare. 

You must obey the following constraints:

{joke_constraints}

""".strip()



from pydantic import BaseModel,Field

class JokeResponse(BaseModel):
    joke_instructions: str = Field(..., description="instruction, not in the style of Shakespeare")     
    joke: str = Field(..., description="joke in the style of Shakespeare")
    joke_explanation: str = Field(..., description="explanation, not in the style of Shakespeare")
    def format(self) -> str:
        return "\n".join([
            "**Joke instructions:**",
            "",
            self.joke_instructions,
            "",
            "**Joke:**",
            "",
            self.joke,
            "",
            "**Joke explanation:**",
            "",
            self.joke_explanation
        ])


default_llm_config = {'cache_seed': 42,
                     'temperature': 1.,
                     'top_p': 0.05,
                     'config_list': [{'model': 'gpt-4o',
                                      'api_key': os.getenv('OPENAI_API_KEY'),
                                      'api_type': 'openai'}],
                     'timeout': 1200}

joker_config_list = copy.deepcopy(default_llm_config)
joker_config_list['config_list'][0]['response_format'] = JokeResponse


joker =  ConversableAgent(
    name="joker",
    system_message=joker_message,
    # llm_config=LLMConfig(model="gpt-4o", 
    #                      api_type="openai",
    #                      response_format=JokeResponse
    #                     ),
    llm_config = joker_config_list,
    update_agent_state_before_reply=[UpdateSystemMessage(joker_message),],
)

workflow_context = ContextVariables(data={
    "joke_constraints": "the joke should make use of the contextual information passed on to you. It should be a paragraph long and use as much detailed information from the context as possible.",
})


task = """
Make a joke based on the title and abstract of an arxiv paper of your choice.
"""

initial_agent = joker

In [4]:
async def create_toolkit_and_run(session: ClientSession) -> None:
    # Create a toolkit with available MCP tools
    toolkit = await create_toolkit(session=session)
    mcp_agent = ConversableAgent(name="mcp_agent", 
                             system_message=r"""
Download arxiv paper and extract titles and abstracts. 
                             """,
                             llm_config=LLMConfig(model="gpt-4o", 
                                                  api_type="openai",
                                                  tool_choice="required"
                                                 ))
    # Register MCP tools with the agent
    toolkit.register_for_llm(mcp_agent)
    
    toolkit.register_for_execution(mcp_agent)

    # joker.handoffs.set_after_work(AgentTarget(mcp_agent))
    joker.handoffs.set_after_work(TerminateTarget())
    
    mcp_agent.handoffs.set_after_work(AgentTarget(joker))


    mcp_agent.handoffs.add_llm_conditions([
            OnCondition(
                target=AgentTarget(joker),
                condition=StringLLMCondition(prompt="The papers have been downloaded."),
                # available=StringAvailableCondition(context_variable="requires_login"),
            ),
    ])




    agents=[joker,
            mcp_agent,
               ]
    
    for agent in agents:
        agent.reset()
    print("all agents reset")

    import shutil
    import os
    
    def delete_cache_folder():
        cache_path = os.path.join(os.getcwd(), ".cache")
        if os.path.isdir(cache_path):
            shutil.rmtree(cache_path)
            print(".cache folder deleted.")
        else:
            print("No .cache folder found in current directory.")
    
    delete_cache_folder()

    # Create the pattern
    agent_pattern = DefaultPattern(
      agents=[joker, mcp_agent],
      initial_agent=mcp_agent,
      context_variables=workflow_context,
    )
    

    await a_initiate_group_chat(
            pattern=agent_pattern,
            messages=task,
            max_rounds=20,
        )



server_params = StdioServerParameters(
    command="python",
    args=[str(mcp_server_path), "stdio", "--storage-path", "arxiv_papers"]
)

async with stdio_client(server_params) as (read, write), ClientSession(read, write) as session:
    # Initialize the connection
    await session.initialize()
    await create_toolkit_and_run(session)

all agents reset
.cache folder deleted.
_User (to chat_manager):


Make a joke based on the title and abstract of an arxiv paper of your choice.


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
mcp_agent (to chat_manager):

***** Suggested tool call (call_sviIr1EV6z6JE8YRJI0yfET3): search_arxiv *****
Arguments: 
{"query":"quantum computing","max_results":1}
*****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search_arxiv...
Call ID: call_sviIr1EV6z6JE8YRJI0yfET3
Input arguments: {'query': 'quantum computing', 'max_results': 1}
_Group_Tool_Executor (to chat_manager):

***** Response from calling tool (call_sviIr1EV6z6JE8YRJI0yfET3) *****
('2208.00733v1', None)
**********************************************************************

-----------------------------